In [16]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [20]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [21]:
tf.test.is_built_with_cuda()

False

In [25]:
pip show tensorflow-gpu

Note: you may need to restart the kernel to use updated packages.


### https://keras.io/api/datasets/cifar10/

In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [5]:
X_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [6]:
X_test.shape, y_test.shape

((10000, 32, 32, 3), (10000, 1))

In [7]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
       dtype=int64))

# Data Manipulation

In [8]:
X_train_scaled = X_train / 255
X_test_scaled = X_test /255

In [9]:
y_train_encoded = keras.utils.to_categorical(y_train, num_classes=10)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes=10)

y_train_encoded.shape, y_train_encoded

((50000, 10),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]))

# Build Model

In [10]:
def nn_adam():
    model = keras.Sequential([
        keras.layers.Input(shape=(32, 32, 3)),  # 32x32x3 = 3072
        keras.layers.Flatten(),
        keras.layers.Dense(300, activation='relu'),
        keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model

In [11]:
def nn_sgd():
    model = keras.Sequential([
        keras.layers.Input(shape=(32, 32, 3)),  # 32x32x3 = 3072
        keras.layers.Flatten(),
        keras.layers.Dense(300, activation='relu'),
        keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer='SGD',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model

# Train Model on CPU

### Bài này chuẩn phải dùng CNN nhé. Dùng NN để test thôi nka!

In [12]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/CPU:0'):
    model_on_CPU = nn_sgd()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2848 - loss: 1.9820
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.3896 - loss: 1.7234
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4242 - loss: 1.6350
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4480 - loss: 1.5760
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4578 - loss: 1.5341
54.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/CPU:0'):
    model_on_CPU = nn_adam()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 14ms/step - accuracy: 0.2851 - loss: 2.0343
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.3799 - loss: 1.7351
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 16ms/step - accuracy: 0.4119 - loss: 1.6568
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 16ms/step - accuracy: 0.4227 - loss: 1.6138
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.4381 - loss: 1.5844
2min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


---

# Train Model on GPU

In [14]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/GPU:0'):
    model_on_CPU = nn_sgd()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.2883 - loss: 1.9764
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.3949 - loss: 1.7185
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4248 - loss: 1.6298
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4479 - loss: 1.5767
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.4661 - loss: 1.5306
53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/GPU:0'):
    model_on_CPU = nn_adam()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 23s 13ms/step - accuracy: 0.2802 - loss: 2.0203
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.3788 - loss: 1.7362
Epoch 3/5
  85/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.4173 - loss: 1.6474Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_7748\4001007766.py", line 1, in <module>
    get_ipython().run_cell_magic('timeit', '-n1 -r1  # to display the time it took to run', "with tf.device('/GPU:0'):\n    model_on_CPU = nn_adam()\n    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)\n")
  File "C:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2517, in run_cell_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\magics\execution.py", line 1189, in timeit
    all_runs = timer.repeat(repeat, number)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\timeit.py", line 208, in repeat
    t = self.timeit(number)
        ^^^^^^^^^^